Chen Zihao 915490404

# Problem 1. Ridge Regression

## 1. Write the gradient descent algorithm with fixed step size for solving (1).

$$
w^* = \arg\min_w\{\frac 1 n \sum_{i=1}^n(x_i^Tw-y_i)^2+\frac\lambda 2 ||w||^2\} := f(w) \eqno{(1)}
$$

f(w) can be also writen as

$$
f(w) = \frac 1 n (y-Xw)^T(y-Xw) +\frac\lambda 2 w^Tw  
$$

$$
\triangledown f = -\frac 2 n (X^Ty)  + (\frac 2 n X^TX+\lambda  I) w
$$

In [1]:
from sklearn import datasets
from sklearn.preprocessing import scale
from scipy import sparse
import numpy as np
import pandas as pd
filename = "./cpusmall"
X,Y = datasets.load_svmlight_file(filename)
X_array = sparse.csr_matrix.todense(X)

#standardized X and y to get rid of the NaN issue.
x = scale(X_array)
y = scale(Y)
    
def GDFSS(eta,X,y):
    """
    Gradient Descent with Fixed Step Size
    """
    #set w0
    p = X.shape[1]
    n = X.shape[0]
    epsilon = 0.001
    w = np.zeros(p)
        
    #for every iteration we need to calculate 2X^TX/n+I 
    #and -2/n*X^Ty, store it to speed up.
    XTX = X.T.dot(X)*2/n+np.eye(p)
    XY = -2/n*X.T.dot(y)
    
    def f1(w):
        return XY+XTX.dot(w)
    
    g = f1(w)
    r0 = np.linalg.norm(g)
    for i in range(200):
        if np.linalg.norm(g)<epsilon*r0:
            break 
        g = f1(w)
        w = w - eta*g
    return w

result=np.zeros([6,13])
for i in range(2,8):
    result[i-2,0] = 10**-i
    result[i-2,1:] = GDFSS(10**-i,x,y)
pd.DataFrame(result)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1.000000e-02,-0.035365,-0.006296,-0.032874,-0.036778,-0.024605,-0.137983,-0.071508,-0.066856,-0.054443,-0.320106,-0.006862,0.302082
1,1.000000e-03,-0.026040,-0.017017,-0.049686,-0.050686,-0.038322,-0.077378,-0.060021,-0.061121,-0.053329,-0.170709,0.042530,0.165858
2,1.000000e-04,-0.005172,-0.004010,-0.011641,-0.011950,-0.009727,-0.013480,-0.010702,-0.012064,-0.010593,-0.024124,0.009798,0.025637
3,1.000000e-05,-0.000560,-0.000440,-0.001279,-0.001315,-0.001077,-0.001442,-0.001145,-0.001305,-0.001146,-0.002508,0.001071,0.002698
4,1.000000e-06,-0.000057,-0.000044,-0.000129,-0.000133,-0.000109,-0.000145,-0.000115,-0.000132,-0.000116,-0.000252,0.000108,0.000271
5,1.000000e-07,-0.000006,-0.000004,-0.000013,-0.000013,-0.000011,-0.000015,-0.000012,-0.000013,-0.000012,-0.000025,0.000011,0.000027


As shown above, the first collumns is the $\eta$ and the remain of each row is the $w$ after 200 iterations.

I compared it as the result of 20000 iterations to see the stability. It seems the $10^{-2}$ is the best which is already around the stable results.

## 2.Do 5-fold cross validation. Each time using 4 of them as training and 1 of them as testing to get MSE, and then report the average MSE.

$$
MSE = \frac 1 n \sum_{i=1}^n (x_i^Tw-y_i)^2 = \frac 1 n (Xw-y)^T(Xw-y)
$$

In [2]:
def MSE(w,X,y):
    n = X.shape[0]
    e = X.dot(w)-y
    return e.T.dot(e)/n

In [3]:
import random

x = pd.DataFrame(x)
y = pd.DataFrame(y)

# get the number of rows
n = X_array.shape[0]

#to get a random list
t = np.arange(n)
random.shuffle(t)

MSE_sum = 0

for i in range(5):
    #get the validation
    test = t[i*round(n/5):(i+1)*round(n/5)]
    x_test = x.iloc[test,]
    y_test = y.iloc[test]
    x_train = x.drop(test)
    y_train = y.drop(test)
    #to get the w
    w = GDFSS(10**-2,x_train,y_train)
    #get the MSE
    MSE_sum = MSE_sum + MSE(w,x_test,y_test).iloc[0,0]
MSE_sum/5

0.33792027939007935

## 3. Run gradient descent on "E2006-tfidf" data. Run your gradient descent implementation. 

In [4]:
filename = "./E2006.train.bz2"
X_train,Y_train = datasets.load_svmlight_file(filename)
filename = "./E2006.test.bz2"
X_test,Y_test = datasets.load_svmlight_file(filename)

As it is a very large sparse matrix, i need to modify the algorithm in 1.1 to make it quicker.

$$
\triangledown f = -\frac 2 n (X^Ty)  + \frac 2 n X^TXw+\lambda  w
$$

In [5]:
def GDFSS2(eta,X,y):
    """
    Gradient Descent with Fixed Step Size
    """
    #set w0
    p = X.shape[1]
    n = X.shape[0]
    epsilon = 0.001
    w = np.zeros(p)
        
    #for every iteration we need to calculate -2/n*X^Ty
    #store it to speed up.
    XY = -2*X.T@(y/n)
    
    def f1(w):
        return XY+2*X.T@(X@(w/n))+w
    
    g = f1(w)
    r0 = np.linalg.norm(g)
    for i in range(200):
        if np.linalg.norm(g)<epsilon*r0:
            break 
        g = f1(w)
        w = w - eta*g
    return w



In [6]:
def MSE(w,X,y):
    n = X.shape[0]
    e = X@w - y
    return e.T@e/n

w = GDFSS2(5*10**-2,X_train,Y_train)
MSE(w[:-2],X_test,Y_test)

0.15098585179963231

# Problem 2. Classification

$$
w^* = \arg\min_w \{\frac 1 n \sum_{i=1}^n \log (1+e^{-y_iw^Tx_i})+\frac \lambda 2 ||w||^2\}:= f(w) \eqno(2)
$$

## 2.1. Derive the gradient of (2)

$$
\begin{split}
\triangledown f(w) &=\frac 1 n \sum_{i=1}^n \frac {1} {1+e^{-y_iw^Tx_i}}\times e^{-y_iw^Tx_i}\times (-y_ix_i)+\lambda w\\
&= -\frac 1 n \sum_{i=1}^n \frac {y_ix_i}{1+e^{y_iw^Tx_i}}+\lambda w\\
\end{split}
$$

In a matrix form.
$$
\triangledown f(w)= -\frac 1 n \frac{X^Ty}{1+e^{W^TX^Ty}}+\lambda w
$$

## 2.2. Implement gradient descent with fixed step size to solve (2). Split it into 80% training and 20% testing. Solve the logistic regression probelm using $\lambda=1$ on the training set, and report the prediction accuracy on test set.

In [7]:
filename = "./news20.binary.bz2"
X,Y = datasets.load_svmlight_file(filename)

Split the dataset into 80% training and 20% testing.

In [8]:
n = X.shape[0]
#to get a random list
t = np.arange(n)
random.shuffle(t)

Y_train,X_train = Y[t[:round(0.8*n)]],X[t[:round(0.8*n)],]
Y_test,X_test = Y[t[round(0.8*n):]],X[t[round(0.8*n):],]

In [10]:
import math
def Logi(eta,X,y):
    """
    Gradient Descent with Fixed Step Size 
    """
    #set w0
    p = X.shape[1]
    n = X.shape[0]
    epsilon = 0.001
    w = np.zeros(p)
        
    #for every iteration we need to calculate -2/n*X^Ty
    #store it to speed up.
    XY = -2*X.T@(y/n)
    
    def f1(w,x,y):
        return -x.T@y/((1+math.exp(w.T@x.T@y))*x.shape[0])+w
    
    g = f1(w,X,y)
    r0 = np.linalg.norm(g)
    for i in range(200):
        if np.linalg.norm(g)<epsilon*r0:
            break 
        g = f1(w,X,y)
        w = w - eta*g
    return w

In [11]:
w = Logi(10**-2,X_train,Y_train)

In [12]:
def predict(w,x):
    s = x@w
    n =s.shape[0]
    y = np.zeros(x.shape[0])
    for i in range(n):
        y[i] = 1/(1+math.exp(-s[i]))
    y = np.sign(y-0.5+10**-15)#to get rid of those exact 0.5
    return y

2.2.2 Accuracy

Here is the confusion matrix for logistic regression and we can use it to calcurate the accuracy.

In [13]:
from sklearn import metrics
confusion = metrics.confusion_matrix(predict(w,X_test),Y_test)
confusion

array([[1831,  656],
       [ 202, 1310]], dtype=int64)

In [14]:
confusion[1,0]+confusion[0,1]

858

In [15]:
(confusion[0,0]+confusion[1,1])/sum(sum(confusion))

0.7854463615903976

So that I got about 850 wrong and i got about 79% accuracy.(change as the the split change)